In [3]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM  

In [4]:
tokenizer = AutoTokenizer.from_pretrained("llmware/bling-1.4b-0.1")  



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model = AutoModelForCausalLM.from_pretrained("llmware/bling-1.4b-0.1")  



In [6]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

bs4_strainer = bs4.SoupStrainer(class_=("main-content"))
loader = WebBaseLoader(
    web_paths=("https://deltek.com/en","https://www.deltek.com/en/about/contact-us", "https://www.deltek.com/en/small-business", "https://www.deltek.com/en/customers",
               "https://www.deltek.com/en/support", "https://www.deltek.com/en/partners"),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()



text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
all_splits_text = [split.page_content for split in all_splits]



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from sentence_transformers import SentenceTransformer
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings


# doc_embeddings = SentenceTransformer('all-MiniLM-L6-v2').encode(all_splits_text)
# query = "Who trusts Deltek?"
# query_embeddings = SentenceTransformer('all-MiniLM-L6-v2').encode(query)
# # print(doc_embeddings.shape)
# pairs = zip(all_splits_text, doc_embeddings)

# # vectorstore = FAISS.from_embeddings(pairs, SentenceTransformer('all-MiniLM-L6-v2'))



model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    # query_instruction="What is Deltek?"
)

query = "Who trusts Deltek?"



In [9]:
# from sklearn.metrics.pairwise import cosine_similarity

# results = cosine_similarity(doc_embeddings, query_embeddings.reshape(1,-1)).reshape((-1,))
# k = 4
# ixs = results.argsort()
# ixs = ixs[::-1]

# relevant_docs = []
# results2 = []
# for i in ixs:
#     relevant_docs.append(all_splits[i].page_content)
#     results2.append(results[i])
# relevant_docs = relevant_docs[:k]
# formatted_docs = "\n\n".join(doc for doc in relevant_docs)

db = FAISS.from_documents(all_splits, embeddings)
query = "How many customers trust Deltek?"
relevant_docs = db.similarity_search(query)
formatted_docs = '\n'.join(doc.page_content for doc in relevant_docs)
print(formatted_docs)


Home

Customers






Customer Stories
Trusted by 30,000 organizations around the world.









Our customers are transforming the world around us












Small Business Boosts Economic Development with Deltek Costpoint.










Firm Future-Proofs Business with Deltek Pros™ and Vantagepoint.










Mission Solutions Group Transforms Operations and Improves Workflow.




















Explore Deltek Customer Success Stories






















Product






Ajera
                                



ArchiSnapper
                                



ComputerEase
                                



Costpoint
                                



Deltek Payments
                                



Govwin IQ
                                



Maconomy
                                



Project Information Management
                                



Project Portfolio Management
                                



ProPricer
Powering Project Success
Deltek is always looking for

In [10]:
context = formatted_docs
print(len(relevant_docs))

4


In [11]:
entries = {"context": context, 
          "query":query}


In [12]:
new_prompt = "<human>: " + entries["context"] + "\n" + entries["query"] + "\n" + "<bot>:"

inputs = tokenizer(new_prompt, return_tensors="pt")  
start_of_output = len(inputs.input_ids[0])

In [13]:
outputs = model.generate(
        inputs.input_ids.to("cpu"),
        attention_mask=inputs.attention_mask.to("cpu"),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.3,
        max_new_tokens=100,
        )


In [14]:
output_text = tokenizer.decode(outputs[0][start_of_output:], skip_special_tokens=True)
print(output_text)

 1,000,000 customers.
